<a href="https://colab.research.google.com/github/seonghunYang/face-recognition/blob/main/%ED%95%99%EC%8A%B5_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%AA%A8%EB%93%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Jul  6 07:21:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/I-mind-training-modules/
!pip install mxnet

Mounted at /content/drive
/content/drive/MyDrive/I-mind-training-modules


In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras
import train, losses, models

In [ ]:
# permission 허용
!chmod 755 -R /content/drive/MyDrive/I-mind-training-modules/

KeyboardInterrupt: ignored

#데이터 셋 생성

In [ ]:
# Convert `/datasets/faces_emore` to `/datasets/faces_emore_112x112_folders`
!CUDA_VISIBLE_DEVICES='-1' ./prepare_data.py -D ./datasets/faces_webface_112x112
# Convert evaluating bin files
!CUDA_VISIBLE_DEVICES='-1' ./prepare_data.py -D ./datasets/faces_webface_112x112 -T lfw.bin cfp_fp.bin agedb_30.bin


save_dir = ./datasets/faces_webface_112x112_112x112_folders, idx_path = ./datasets/faces_webface_112x112/train.idx, bin_path = ./datasets/faces_webface_112x112/train.rec
./datasets/faces_webface_112x112_112x112_folders already exists.
2021-07-06 14:50:46.596764: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
test_bins = ['./datasets/faces_webface_112x112/lfw.bin', './datasets/faces_webface_112x112/cfp_fp.bin', './datasets/faces_webface_112x112/agedb_30.bin']
Saving to ./datasets/faces_webface_112x112/lfw.bin
Saving to ./datasets/faces_webface_112x112/cfp_fp.bin
Saving to ./datasets/faces_webface_112x112/agedb_30.bin
save_dir = ./datasets/faces_webface_112x112_112x112_folders, idx_path = ./datasets/faces_webface_112x112/train.idx, bin_path = ./datasets/faces_webface_112x112/train.rec
./datasets/faces_webface_112x112_112x112_folders already exists.


In [ ]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


#추가 학습

In [ ]:
data_basic_path = './datasets/'
data_path = data_basic_path + 'faces_webface_112x112_112x112_folders'
eval_paths = [data_basic_path + ii for ii in ['faces_webface_112x112/lfw.bin', 'faces_webface_112x112/cfp_fp.bin', 'faces_webface_112x112/agedb_30.bin']]

pre_train_model = "resnet34_MXNET_E_SGD_REG_1e3_on_batch_lr1e1_random0_arc_S32_E1_BS512_casia_basic_agedb_30_epoch_35_0.947500.h5"

basic_model = models.buildin_models("r34", dropout=0.4, emb_shape=512, output_layer='E', bn_momentum=0.9, bn_epsilon=2e-5, use_bias=True, scale=False)
basic_model= models.replace_ReLU_with_PReLU(basic_model)
basic_model = models.add_l2_regularizer_2_model(basic_model, 1e-3, apply_to_batch_normal=True)
tt = train.Train(data_path, save_path='./test.h5',
    eval_paths=eval_paths, basic_model=pre_train_model, model=None, lr_base=0.1, lr_decay=0.1, lr_decay_steps=[20, 30],
    batch_size=16, random_status=0, output_weight_decay=1)

# optimizer = tfa.optimizers.SGDW(learning_rate=0.1, weight_decay=5e-4, momentum=0.9)


Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_run.py", line 183, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 755, in communicate_network_status
    resp = self._communicate(req, timeout=timeout, local=True)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 545, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 550, in _communicate_async
    raise Exception("The wandb backend process has shutdown")
Exception: The wandb backe

>>>> Change BatchNormalization momentum and epsilon default value.
>>>> Convert ReLU: 0_relu --> 0_prelu
>>>> Convert ReLU: stack1_block1_2_relu --> stack1_block1_2_prelu
>>>> Convert ReLU: stack1_block2_2_relu --> stack1_block2_2_prelu
>>>> Convert ReLU: stack1_block3_2_relu --> stack1_block3_2_prelu
>>>> Convert ReLU: stack2_block1_2_relu --> stack2_block1_2_prelu
>>>> Convert ReLU: stack2_block2_2_relu --> stack2_block2_2_prelu
>>>> Convert ReLU: stack2_block3_2_relu --> stack2_block3_2_prelu
>>>> Convert ReLU: stack2_block4_2_relu --> stack2_block4_2_prelu
>>>> Convert ReLU: stack3_block1_2_relu --> stack3_block1_2_prelu
>>>> Convert ReLU: stack3_block2_2_relu --> stack3_block2_2_prelu
>>>> Convert ReLU: stack3_block3_2_relu --> stack3_block3_2_prelu
>>>> Convert ReLU: stack3_block4_2_relu --> stack3_block4_2_prelu
>>>> Convert ReLU: stack3_block5_2_relu --> stack3_block5_2_prelu
>>>> Convert ReLU: stack3_block6_2_relu --> stack3_block6_2_prelu
>>>> Convert ReLU: stack4_block1_2_re

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 40},
]
tt.train(sch, 0)

In [ ]:
!pip install wandb
import wandb
wandb.init()